# Three approaches to a market basket analysis
 This notebook shows three different ways of identifying items that are commonly purchased together. The goal is to come up with the same answer, but compare the complexity of different ways to implement the analysis:
1. SQL
2. Dataframes in Snowpark for Python
3. Using a Python package called `efficient-apriori`
    1. On your local machine (or notebook server).
    2. In a Python stored procedure in Snowflake.

The data used in this analysis is the same as is used in the [Snowflake Tasty Bytes Quickstarts](https://quickstarts.snowflake.com/?cat=tasty-bytes). You can use this data in your own Snowflake account or a trial acount using [this code](https://github.com/Snowflake-Labs/sf-samples/blob/main/samples/tasty_bytes/tb_introduction.sql).


All three market basket analyses implement the [Apriori algorithm](https://en.wikipedia.org/wiki/Apriori_algorithm), which is possibly the most popular algorithm used for association rule mining. For this analysis, we'll refer the the primary item as MENU_ITEM_NAME (sometimes called Left Hand Side, LHS, A), and added item as MENU_ITEM_NAME_B (sometimes called Right Hand side, RHS, B). With hese items, we'll calculate the following:
* **Associated Orders** - Orders with both items A and B 
* **Item Total Orders** - Total number of orders with Item A 
* **Item B Total Orders** - Total number of orders with Item B
* **Total Orders** - Total number of orders in dataset
* **Attachement Percent** - Percent of Item A orders also containing Item B -- P(B|A)
* **Expected Attachment Percent** Attachement Percent - Percent of Total Orders containing Item B -- P(B)
* **Lift** - Attachment Percent / Expected Attachement percent -- P(B|A)/P(B)
* **Support** - P(AB) 

In [1]:
# Snowpark for Python
from snowflake.snowpark.session import Session
from snowflake.snowpark.dataframe import DataFrame
from snowflake.snowpark.functions import countDistinct, col, sproc, listagg
from snowflake.snowpark.version import VERSION
# Misc
import json 
import pandas as pd
import logging 
from IPython.display import display, Markdown, Latex

logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)
pd.set_option = ('precision', 4)

In [2]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(snowflake_environment[0][1]))
print('Database                    : {}'.format(snowflake_environment[0][2]))
print('Schema                      : {}'.format(snowflake_environment[0][3]))
print('Warehouse                   : {}'.format(snowflake_environment[0][5]))
print('Snowflake version           : {}'.format(snowflake_environment[0][4]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

User                        : JGRIFFITH
Role                        : SYSADMIN
Database                    : FROSTBYTE_TASTY_BYTES
Schema                      : RAW_POS
Warehouse                   : LARGE
Snowflake version           : 7.10.1
Snowpark for Python version : 1.1.0


## SQL Approach
This is how you would accomplish this market basket analysis using SQL, and the way I've done this 100 times before. It works well, and using Common Table Expression makes the code more readable and managable and takes 54 lines of code. 

In [3]:
pd_df = session.sql(
    """
    with order_detail_menu as (
        select distinct
            b.menu_item_name,
            a.order_id
        from 
            order_detail a
            inner join menu b
                on a.menu_item_id = b.menu_item_id
                and b.truck_brand_name = 'Guac n\\\' Roll'
    ),
    associations as (
        select
            a.menu_item_name,
            b.menu_item_name as menu_item_name_b,
            count(distinct a.order_id) as associated_orders
        from 
            order_detail_menu a
            inner join order_detail_menu b
                on a.order_id = b.order_id
                and a.menu_item_name != b.menu_item_name
        group by 
            a.menu_item_name,
            b.menu_item_name
    ),
    base_frequencies as (
        select
            menu_item_name,
            count(distinct order_id) as item_total_orders
        from 
            order_detail_menu
        group by 
            menu_item_name
    ),
    total_orders as (
        select 
            count(distinct order_id) as total_orders
        from 
            order_detail_menu
    )
    select
        a.*,
        b.item_total_orders,
        c.item_total_orders as item_b_total_orders,
        d.total_orders,
        a.associated_orders / b.item_total_orders as attachment_pct,
        c.item_total_orders / d.total_orders as expected_attachment_pct,
        attachment_pct / expected_attachment_pct as lift
    from 
        associations a
        inner join base_frequencies b
            on a.menu_item_name = b.menu_item_name
        inner join base_frequencies c
            on a.menu_item_name_b = c.menu_item_name
        inner join total_orders d
    """
).to_pandas()

In [4]:
pd_df.sort_values('LIFT', ascending=False).iloc[:10]

,MENU_ITEM_NAME,MENU_ITEM_NAME_B,ASSOCIATED_ORDERS,ITEM_TOTAL_ORDERS,ITEM_B_TOTAL_ORDERS,TOTAL_ORDERS,ATTACHMENT_PCT,EXPECTED_ATTACHMENT_PCT,LIFT
5,Ice Tea,Bottled Water,1115928,3264243,3264528,15827030,0.341864,0.206263,1.657417956686
21,Bottled Water,Ice Tea,1115928,3264528,3264243,15827030,0.341834,0.206245,1.657417149507
27,Bottled Soda,Ice Tea,1115258,3264192,3264243,15827030,0.341664,0.206245,1.656592887100
1,Ice Tea,Bottled Soda,1115258,3264243,3264192,15827030,0.341659,0.206242,1.656592740567
24,Bottled Soda,Bottled Water,1114911,3264192,3264528,15827030,0.341558,0.206263,1.655934413831
22,Bottled Water,Bottled Soda,1114911,3264528,3264192,15827030,0.341523,0.206242,1.655933321050
54,Fish Burrito,Ice Tea,1890317,6692795,3264243,15827030,0.282441,0.206245,1.369444107736
4,Ice Tea,Fish Burrito,1890317,3264243,6692795,15827030,0.579098,0.422871,1.369443636475
19,Bottled Water,Chicken Burrito,1891175,3264528,6696381,15827030,0.579310,0.423098,1.369209970267
9,Chicken Burrito,Bottled Water,1891175,6696381,3264528,15827030,0.282417,0.206263,1.369208243844


Even if we use SQL for implementing the apriori algorithm, we can still pull the result to Python for display and further exploration. The Pandas pivot function is more concise than using a SQL pivot command or a series of sum-case statements in SQL.

In [5]:
display(Markdown("### Attachment Percents"))
display(pd_df.pivot(index='MENU_ITEM_NAME', columns='MENU_ITEM_NAME_B')['ATTACHMENT_PCT'].fillna(''))

display(Markdown("### Lift"))
display(pd_df.pivot(index='MENU_ITEM_NAME', columns='MENU_ITEM_NAME_B')['LIFT'].fillna(''))

### Attachment Percents

MENU_ITEM_NAME_B,Bottled Soda,Bottled Water,Chicken Burrito,Fish Burrito,Ice Tea,Lean Burrito Bowl,Three Taco Combo Plate,Two Taco Combo Plate,Veggie Taco Bowl
MENU_ITEM_NAME,,,,,,,,,
Bottled Soda,,0.341558,0.579020,0.578628,0.341664,0.579017,0.578678,0.578345,0.578288
Bottled Water,0.341523,,0.579310,0.578695,0.341834,0.578354,0.578730,0.578854,0.578520
Chicken Burrito,0.282247,0.282417,,0.436959,0.281979,0.437018,0.436953,0.437272,0.436828
Fish Burrito,0.282207,0.282268,0.437194,,0.282441,0.437099,0.436816,0.436856,0.436916
Ice Tea,0.341659,0.341864,0.578462,0.579098,,0.578969,0.578440,0.578120,0.578815
Lean Burrito Bowl,0.282337,0.282043,0.437160,0.437006,0.282318,,0.437017,0.436809,0.437024
Three Taco Combo Plate,0.282197,0.282251,0.437134,0.436762,0.282085,0.437057,,0.436875,0.437097
Two Taco Combo Plate,0.282040,0.282318,0.437463,0.436812,0.281935,0.436858,0.436884,,0.436880
Veggie Taco Bowl,0.282019,0.282161,0.437027,0.436882,0.282280,0.437082,0.437116,0.436890,


### Lift

MENU_ITEM_NAME_B,Bottled Soda,Bottled Water,Chicken Burrito,Fish Burrito,Ice Tea,Lean Burrito Bowl,Three Taco Combo Plate,Two Taco Combo Plate,Veggie Taco Bowl
MENU_ITEM_NAME,,,,,,,,,
Bottled Soda,,1.655934413831,1.368524549868,1.368332186411,1.656592887100,1.368960731604,1.368282169568,1.367523893747,1.367421447894
Bottled Water,1.655933321050,,1.369209970267,1.368490627165,1.657417149507,1.367393211194,1.368405123391,1.368727448134,1.367970035753
Chicken Burrito,1.368523385149,1.369208243844,,1.033315124471,1.367204053432,1.033234742683,1.033173887445,1.033950164809,1.032924729962
Fish Burrito,1.368329438233,1.368485865133,1.033316158431,,1.369444107736,1.033426249702,1.032849951410,1.032966513286,1.033132815012
Ice Tea,1.656592740567,1.657417956686,1.367205706479,1.369443636475,,1.368847245964,1.367719419374,1.366991870688,1.368667593591
Lean Burrito Bowl,1.368959765712,1.367395024798,1.033235798798,1.033426269477,1.368847729642,,1.033325215228,1.032855379581,1.033388192119
Three Taco Combo Plate,1.368280951504,1.368403446086,1.033174347314,1.032849261359,1.367718005285,1.033326949766,,1.033011439678,1.033560808127
Two Taco Combo Plate,1.367519709855,1.368728274097,1.033951944939,1.032967500727,1.366990714926,1.032856457215,1.033010737179,,1.033047689310
Veggie Taco Bowl,1.367417887724,1.367967109952,1.032921450822,1.033133035843,1.368663482751,1.033386056870,1.033559300393,1.033046907882,


## Snowpark for Python dataframe approach
This next section will perform the same caluclations as the sql statement above, but will use Snowpark Python dataframes to complete the calculations. This apporach uses 20 lines of code, and also gives us logical places to include comments. With comments and spacing, this is 42 lines of code. 

In [6]:
# get distinct order_id/menu_item_id combinations, join to menu item names
# filter for one specific food truck brand
order_detail = session.table('ORDER_DETAIL').select(['ORDER_ID','MENU_ITEM_ID']).distinct()
menu = session.table('MENU').filter(col('TRUCK_BRAND_NAME') == "Guac n' Roll").select(['MENU_ITEM_ID', 'MENU_ITEM_NAME'])
order_detail = order_detail.join(menu, on='MENU_ITEM_ID')

# create duplicate order_detail_dataframe and self join on order_id to 
# get our product to product associations
order_detail_b = order_detail.select([col(c).alias(c+'_B') for c in order_detail.columns])
associations = order_detail.join(order_detail_b, on=order_detail.ORDER_ID == order_detail_b.ORDER_ID_B)

# remove duplicate cases because lift won't be calculated 
pairs = associations.where('MENU_ITEM_NAME != MENU_ITEM_NAME_B')\
                    .group_by(['MENU_ITEM_NAME', 'MENU_ITEM_NAME_B'])\
                    .agg(countDistinct('ORDER_ID').as_('ASSOCIATED_ORDERS'))

# count base occurances of item purchases
base_frequencies = associations.group_by('MENU_ITEM_NAME').agg(countDistinct('ORDER_ID').as_('ITEM_TOTAL_ORDERS'))

# total universe of orders
total_orders = order_detail.agg(countDistinct('ORDER_ID').as_('TOTAL_ORDERS'))

# join total orders for main item
basket_analysis = pairs.join(base_frequencies.select(['MENU_ITEM_NAME' , 'ITEM_TOTAL_ORDERS']), on='MENU_ITEM_NAME' )

# join total orders for associated item (item B)
basket_analysis = basket_analysis.join(base_frequencies.select(col('MENU_ITEM_NAME').as_('MENU_ITEM_NAME_B'),
                                                               col('ITEM_TOTAL_ORDERS').as_('ITEM_B_TOTAL_ORDERS')
                                                              ), 
                                       on='MENU_ITEM_NAME_B' )
                        
# join total orders
basket_analysis = basket_analysis.join(total_orders)

basket_analysis = basket_analysis.select(['MENU_ITEM_NAME', 'MENU_ITEM_NAME_B', 'ASSOCIATED_ORDERS', 'ITEM_TOTAL_ORDERS','ITEM_B_TOTAL_ORDERS','TOTAL_ORDERS']) \
                                 .withColumn('ATTACHMENT_PCT', col('ASSOCIATED_ORDERS') / col('ITEM_TOTAL_ORDERS')) \
                                 .withColumn('EXPECTED_ATTACHMENT_PCT', col('ITEM_B_TOTAL_ORDERS') / col('TOTAL_ORDERS')) \
                                 .withColumn('LIFT', col('ATTACHMENT_PCT') / col('EXPECTED_ATTACHMENT_PCT')) \
                                 .withColumn('SUPPORT', col('ASSOCIATED_ORDERS') / col('TOTAL_ORDERS'))

In [7]:
pd_df = basket_analysis.to_pandas()

display(Markdown("### Attachement percents"))
display(pd_df.pivot(index='MENU_ITEM_NAME', columns='MENU_ITEM_NAME_B')['ATTACHMENT_PCT'].fillna(''))

display(Markdown("### Lift"))
display(pd_df.pivot(index='MENU_ITEM_NAME', columns='MENU_ITEM_NAME_B')['LIFT'].fillna(''))

### Attachement percents

MENU_ITEM_NAME_B,Bottled Soda,Bottled Water,Chicken Burrito,Fish Burrito,Ice Tea,Lean Burrito Bowl,Three Taco Combo Plate,Two Taco Combo Plate,Veggie Taco Bowl
MENU_ITEM_NAME,,,,,,,,,
Bottled Soda,,0.341558,0.579020,0.578628,0.341664,0.579017,0.578678,0.578345,0.578288
Bottled Water,0.341523,,0.579310,0.578695,0.341834,0.578354,0.578730,0.578854,0.578520
Chicken Burrito,0.282247,0.282417,,0.436959,0.281979,0.437018,0.436953,0.437272,0.436828
Fish Burrito,0.282207,0.282268,0.437194,,0.282441,0.437099,0.436816,0.436856,0.436916
Ice Tea,0.341659,0.341864,0.578462,0.579098,,0.578969,0.578440,0.578120,0.578815
Lean Burrito Bowl,0.282337,0.282043,0.437160,0.437006,0.282318,,0.437017,0.436809,0.437024
Three Taco Combo Plate,0.282197,0.282251,0.437134,0.436762,0.282085,0.437057,,0.436875,0.437097
Two Taco Combo Plate,0.282040,0.282318,0.437463,0.436812,0.281935,0.436858,0.436884,,0.436880
Veggie Taco Bowl,0.282019,0.282161,0.437027,0.436882,0.282280,0.437082,0.437116,0.436890,


### Lift

MENU_ITEM_NAME_B,Bottled Soda,Bottled Water,Chicken Burrito,Fish Burrito,Ice Tea,Lean Burrito Bowl,Three Taco Combo Plate,Two Taco Combo Plate,Veggie Taco Bowl
MENU_ITEM_NAME,,,,,,,,,
Bottled Soda,,1.655934413831,1.368524549868,1.368332186411,1.656592887100,1.368960731604,1.368282169568,1.367523893747,1.367421447894
Bottled Water,1.655933321050,,1.369209970267,1.368490627165,1.657417149507,1.367393211194,1.368405123391,1.368727448134,1.367970035753
Chicken Burrito,1.368523385149,1.369208243844,,1.033315124471,1.367204053432,1.033234742683,1.033173887445,1.033950164809,1.032924729962
Fish Burrito,1.368329438233,1.368485865133,1.033316158431,,1.369444107736,1.033426249702,1.032849951410,1.032966513286,1.033132815012
Ice Tea,1.656592740567,1.657417956686,1.367205706479,1.369443636475,,1.368847245964,1.367719419374,1.366991870688,1.368667593591
Lean Burrito Bowl,1.368959765712,1.367395024798,1.033235798798,1.033426269477,1.368847729642,,1.033325215228,1.032855379581,1.033388192119
Three Taco Combo Plate,1.368280951504,1.368403446086,1.033174347314,1.032849261359,1.367718005285,1.033326949766,,1.033011439678,1.033560808127
Two Taco Combo Plate,1.367519709855,1.368728274097,1.033951944939,1.032967500727,1.366990714926,1.032856457215,1.033010737179,,1.033047689310
Veggie Taco Bowl,1.367417887724,1.367967109952,1.032921450822,1.033133035843,1.368663482751,1.033386056870,1.033559300393,1.033046907882,


## Using the efficient-apriori package
We're going to use the [efficient-apriori](https://pypi.org/project/efficient-apriori/) package to do a simple apriori analysis. First, we'll do this on our local machine. Then, we'll use a [Snowpark-optimized warehouse](https://docs.snowflake.com/en/user-guide/warehouses-snowpark-optimized) (more memory, more cache) because we'll be collecting all of the data onto a single node to pass it to the apriori function. 

Using this package, we get a better analysis that our SQL and dataframe analyses, because it looks at combinations of more than two products. 

### Calculating apriori on local machine
Here we pull all of the transactions to our local machine (or wherever our notebook is running). This may not work for very large transaction volumes. 

This approach takes about 8 lines of code because all of the logic of the algorithm is in the `efficient-apriori` package.

In [8]:
# get distinct order_id/menu_item_id combinations, join to menu item names
# filter for one specific food truck brand
order_detail = session.table('ORDER_DETAIL').select(['ORDER_ID','MENU_ITEM_ID']).distinct()
menu = session.table('MENU').filter(col('TRUCK_BRAND_NAME') == "Guac n' Roll").select(['MENU_ITEM_ID', 'MENU_ITEM_NAME'])
order_detail = order_detail.join(menu, on='MENU_ITEM_ID')

In [9]:
txn = order_detail.group_by('ORDER_ID')\
                  .agg(listagg('MENU_ITEM_NAME', ',').as_('ORDER_ITEMS'))\
                  .collect()
txn_list = [l['ORDER_ITEMS'].split(',') for l in txn]

In [10]:
import efficient_apriori
itemsets, rules = efficient_apriori.apriori(txn_list, min_support = 0.005, min_confidence = 0.01)

In [11]:
pd_df = pd.DataFrame([  [r.lhs,
                         r.rhs,
                         r.count_full,
                         r.count_lhs,
                         r.count_rhs,
                         r.num_transactions,
                         r.confidence, 
                         r.count_rhs / r.num_transactions,
                         r.lift,
                         r.support
                        ] 
                        for r in rules
                    ],
                    columns = ['MENU_ITEM_NAME','MENU_ITEM_NAME_B','ASSOCIATED_ORDERS','ITEM_TOTAL_ORDERS',
                               'ITEM_B_TOTAL_ORDERS','TOTAL_ORDERS','ATTACHMENT_PCT',
                               'EXPECTED_ATTACHMENT_PCT','LIFT','SUPPORT' ]
).sort_values(['MENU_ITEM_NAME','MENU_ITEM_NAME_B'])

In [12]:
pd_df[['MENU_ITEM_NAME','MENU_ITEM_NAME_B','ATTACHMENT_PCT','LIFT']].sort_values('ATTACHMENT_PCT', ascending=False)

,MENU_ITEM_NAME,MENU_ITEM_NAME_B,ATTACHMENT_PCT,LIFT
662,"(Bottled Soda, Bottled Water, Ice Tea)","(Fish Burrito,)",0.694663,1.642730
592,"(Bottled Soda, Bottled Water, Ice Tea)","(Chicken Burrito,)",0.694203,1.640764
747,"(Bottled Soda, Bottled Water, Ice Tea)","(Three Taco Combo Plate,)",0.693821,1.640538
775,"(Bottled Soda, Bottled Water, Ice Tea)","(Veggie Taco Bowl,)",0.693570,1.640017
761,"(Bottled Soda, Bottled Water, Ice Tea)","(Two Taco Combo Plate,)",0.693463,1.639728
...,...,...,...,...
13465,"(Lean Burrito Bowl,)","(Bottled Soda, Bottled Water, Fish Burrito, Ic...",0.012380,1.473202
11954,"(Fish Burrito,)","(Bottled Soda, Bottled Water, Chicken Burrito,...",0.012378,1.470713
11951,"(Veggie Taco Bowl,)","(Bottled Soda, Bottled Water, Chicken Burrito,...",0.012377,1.472355
11952,"(Three Taco Combo Plate,)","(Bottled Soda, Bottled Water, Chicken Burrito,...",0.012377,1.472092


In [13]:
# examine only two-item associations
pd_df_pairs = pd_df.loc[(pd_df['MENU_ITEM_NAME'].apply(len) == 1) & (pd_df['MENU_ITEM_NAME_B'].apply(len) == 1)]

display(Markdown('### Attachment Percent'))
display(pd_df_pairs.pivot(index='MENU_ITEM_NAME', columns='MENU_ITEM_NAME_B')['ATTACHMENT_PCT'].fillna(''))

display(Markdown('### Lift'))
display(pd_df_pairs.pivot(index='MENU_ITEM_NAME', columns='MENU_ITEM_NAME_B')['LIFT'].fillna(''))

### Attachment Percent

MENU_ITEM_NAME_B,"(Bottled Soda,)","(Bottled Water,)","(Chicken Burrito,)","(Fish Burrito,)","(Ice Tea,)","(Lean Burrito Bowl,)","(Three Taco Combo Plate,)","(Two Taco Combo Plate,)","(Veggie Taco Bowl,)"
MENU_ITEM_NAME,,,,,,,,,
"(Bottled Soda,)",,0.341558,0.57902,0.578628,0.341664,0.579017,0.578678,0.578345,0.578288
"(Bottled Water,)",0.341523,,0.57931,0.578695,0.341834,0.578354,0.57873,0.578854,0.57852
"(Chicken Burrito,)",0.282247,0.282417,,0.436959,0.281979,0.437018,0.436953,0.437272,0.436828
"(Fish Burrito,)",0.282207,0.282268,0.437194,,0.282441,0.437099,0.436816,0.436856,0.436916
"(Ice Tea,)",0.341659,0.341864,0.578462,0.579098,,0.578969,0.57844,0.57812,0.578815
"(Lean Burrito Bowl,)",0.282337,0.282043,0.43716,0.437006,0.282318,,0.437017,0.436809,0.437024
"(Three Taco Combo Plate,)",0.282197,0.282251,0.437134,0.436762,0.282085,0.437057,,0.436875,0.437097
"(Two Taco Combo Plate,)",0.28204,0.282318,0.437463,0.436812,0.281935,0.436858,0.436884,,0.43688
"(Veggie Taco Bowl,)",0.282019,0.282161,0.437027,0.436882,0.28228,0.437082,0.437116,0.43689,


### Lift

MENU_ITEM_NAME_B,"(Bottled Soda,)","(Bottled Water,)","(Chicken Burrito,)","(Fish Burrito,)","(Ice Tea,)","(Lean Burrito Bowl,)","(Three Taco Combo Plate,)","(Two Taco Combo Plate,)","(Veggie Taco Bowl,)"
MENU_ITEM_NAME,,,,,,,,,
"(Bottled Soda,)",,1.655936,1.368526,1.368332,1.656596,1.36896,1.368284,1.367524,1.36742
"(Bottled Water,)",1.655936,,1.369212,1.368489,1.657421,1.367394,1.368406,1.368728,1.367969
"(Chicken Burrito,)",1.368526,1.369212,,1.033316,1.367207,1.033236,1.033175,1.033952,1.032923
"(Fish Burrito,)",1.368332,1.368489,1.033316,,1.369443,1.033427,1.03285,1.032968,1.033132
"(Ice Tea,)",1.656596,1.657421,1.367207,1.369443,,1.368847,1.367721,1.366992,1.368667
"(Lean Burrito Bowl,)",1.36896,1.367394,1.033236,1.033427,1.368847,,1.033326,1.032855,1.033387
"(Three Taco Combo Plate,)",1.368284,1.368406,1.033175,1.03285,1.367721,1.033326,,1.033012,1.033559
"(Two Taco Combo Plate,)",1.367524,1.368728,1.033952,1.032968,1.366992,1.032855,1.033012,,1.033047
"(Veggie Taco Bowl,)",1.36742,1.367969,1.032923,1.033132,1.368667,1.033387,1.033559,1.033047,


### Run efficient-apriori in Snowflake compute
We can also load the [efficient-apriori](https://pypi.org/project/efficient-apriori/) package to our Snowflake account and use it in a stored procedure. Because this pulls all the records to a single node, we will use a [Snowpark-optimized Warehouse](https://docs.snowflake.com/en/user-guide/warehouses-snowpark-optimized), which gives us more memory and more cache on a single node. This is the same type of warehouse we would use if we were doing single-node model training. 

First, we'll [download the wheel file from PyPi](https://pypi.org/project/efficient-apriori/#files). Next, we'll create a stored procedure that loads that wheel file and performs the apriori analysis on our Snowpark-optimized warehouse. We'll return the results as an array and load it into a Pandas dataframe for looking at as a crosstab. 

This approach takes about 34 lines of code to implement, but we can also persist the stored procedure to live in the database, so the analysis could be parameterized and re-run easily by anyone who has access to the stored procedure in Snowflake, whether they're using Python or SQL. 

In [14]:
# download efficient-apriori so we can upload it to snowflake
import urllib.request
urllib.request.urlretrieve('https://files.pythonhosted.org/packages/9a/99/f7bda66343a0d535da962b935ec995c2a490ce5f25e57dabf57f035510b0/efficient_apriori-2.0.3-py3-none-any.whl', 'efficient_apriori-2.0.3-py3-none-any.whl')

('efficient_apriori-2.0.3-py3-none-any.whl',
 <http.client.HTTPMessage at 0x7fa3ee6c6730>)

In [15]:
# use a snowpark-optimized warehouse
session.sql('use warehouse snowpark_medium').collect()

[Row(status='Statement executed successfully.')]

In [16]:
# create a stored procedure that loads efficient-apriori
# this allows us to do all of our processing in our Snowflake compute instead of pulling 
# all the data to our local machine
# because our product catalog is small, we can return the result as an array
@sproc(imports=['efficient_apriori-2.0.3-py3-none-any.whl'], 
       packages=['snowflake-snowpark-python'])
def apriori_sproc(session: Session) -> list:
       import sys, zipfile
       
       # unzip and load the efficient_apriori module
       import_dir = sys._xoptions.get("snowflake_import_directory")
       target_dir = '/tmp/efficient_apriori'
       with zipfile.ZipFile(import_dir + "efficient_apriori-2.0.3-py3-none-any.whl", 'r') as zip_file:
              zip_file.extractall(target_dir)

       sys.path.append(target_dir)

       import efficient_apriori
       
       order_detail = session.table('ORDER_DETAIL').select(['ORDER_ID','MENU_ITEM_ID']).distinct()
       menu = session.table('MENU').filter(col('TRUCK_BRAND_NAME') == "Guac n' Roll").select(['MENU_ITEM_ID', 'MENU_ITEM_NAME'])
       order_detail = order_detail.join(menu, on='MENU_ITEM_ID')

       txn = order_detail.group_by('ORDER_ID')\
                  .agg(listagg('MENU_ITEM_NAME', ',').as_('ORDER_ITEMS'))\
                  .collect()
       txn_list = [l['ORDER_ITEMS'].split(',') for l in txn]
       
       itemsets, rules = efficient_apriori.apriori(txn_list, min_support = 0.005, min_confidence = 0.01)

       rules_list = [  [r.lhs,
                         r.rhs,
                         r.count_full,
                         r.count_lhs,
                         r.count_rhs,
                         r.num_transactions,
                         r.confidence, 
                         r.count_rhs / r.num_transactions,
                         r.lift,
                         r.support
                        ] 
                        for r in rules
                    ]
       return rules_list

In [17]:
results = apriori_sproc()

In [18]:
import ast
pd_df = pd.DataFrame(ast.literal_eval(results),
                     columns = ['MENU_ITEM_NAME','MENU_ITEM_NAME_B','ASSOCIATED_ORDERS','ITEM_TOTAL_ORDERS',
                               'ITEM_B_TOTAL_ORDERS','TOTAL_ORDERS','ATTACHMENT_PCT',
                               'EXPECTED_ATTACHMENT_PCT','LIFT','SUPPORT' ]
                    )
pd_df[['MENU_ITEM_NAME','MENU_ITEM_NAME_B']] = pd_df[['MENU_ITEM_NAME','MENU_ITEM_NAME_B']].applymap(tuple)

In [19]:
# examine only two-item associations
pd_df_pairs = pd_df.loc[(pd_df['MENU_ITEM_NAME'].apply(len) == 1) & (pd_df['MENU_ITEM_NAME_B'].apply(len) == 1)]

# display(Markdown('### Attachment Percent'))
display(pd_df_pairs.pivot(index='MENU_ITEM_NAME', columns='MENU_ITEM_NAME_B')['ATTACHMENT_PCT'].fillna(''))

display(Markdown('### Lift'))
display(pd_df_pairs.pivot(index='MENU_ITEM_NAME', columns='MENU_ITEM_NAME_B')['LIFT'].fillna(''))

MENU_ITEM_NAME_B,"(Bottled Soda,)","(Bottled Water,)","(Chicken Burrito,)","(Fish Burrito,)","(Ice Tea,)","(Lean Burrito Bowl,)","(Three Taco Combo Plate,)","(Two Taco Combo Plate,)","(Veggie Taco Bowl,)"
MENU_ITEM_NAME,,,,,,,,,
"(Bottled Soda,)",,0.341558,0.57902,0.578628,0.341664,0.579017,0.578678,0.578345,0.578288
"(Bottled Water,)",0.341523,,0.57931,0.578695,0.341834,0.578354,0.57873,0.578854,0.57852
"(Chicken Burrito,)",0.282247,0.282417,,0.436959,0.281979,0.437018,0.436953,0.437272,0.436828
"(Fish Burrito,)",0.282207,0.282268,0.437194,,0.282441,0.437099,0.436816,0.436856,0.436916
"(Ice Tea,)",0.341659,0.341864,0.578462,0.579098,,0.578969,0.57844,0.57812,0.578815
"(Lean Burrito Bowl,)",0.282337,0.282043,0.43716,0.437006,0.282318,,0.437017,0.436809,0.437024
"(Three Taco Combo Plate,)",0.282197,0.282251,0.437134,0.436762,0.282085,0.437057,,0.436875,0.437097
"(Two Taco Combo Plate,)",0.28204,0.282318,0.437463,0.436812,0.281935,0.436858,0.436884,,0.43688
"(Veggie Taco Bowl,)",0.282019,0.282161,0.437027,0.436882,0.28228,0.437082,0.437116,0.43689,


### Lift

MENU_ITEM_NAME_B,"(Bottled Soda,)","(Bottled Water,)","(Chicken Burrito,)","(Fish Burrito,)","(Ice Tea,)","(Lean Burrito Bowl,)","(Three Taco Combo Plate,)","(Two Taco Combo Plate,)","(Veggie Taco Bowl,)"
MENU_ITEM_NAME,,,,,,,,,
"(Bottled Soda,)",,1.655936,1.368526,1.368332,1.656596,1.36896,1.368284,1.367524,1.36742
"(Bottled Water,)",1.655936,,1.369212,1.368489,1.657421,1.367394,1.368406,1.368728,1.367969
"(Chicken Burrito,)",1.368526,1.369212,,1.033316,1.367207,1.033236,1.033175,1.033952,1.032923
"(Fish Burrito,)",1.368332,1.368489,1.033316,,1.369443,1.033427,1.03285,1.032968,1.033132
"(Ice Tea,)",1.656596,1.657421,1.367207,1.369443,,1.368847,1.367721,1.366992,1.368667
"(Lean Burrito Bowl,)",1.36896,1.367394,1.033236,1.033427,1.368847,,1.033326,1.032855,1.033387
"(Three Taco Combo Plate,)",1.368284,1.368406,1.033175,1.03285,1.367721,1.033326,,1.033012,1.033559
"(Two Taco Combo Plate,)",1.367524,1.368728,1.033952,1.032968,1.366992,1.032855,1.033012,,1.033047
"(Veggie Taco Bowl,)",1.36742,1.367969,1.032923,1.033132,1.368667,1.033387,1.033559,1.033047,
